# How to apply probabilistic error cancellation with Mitiq

In [18]:
import cirq
from mitiq import zne, benchmarks, pec


def execute(circuit: cirq.Circuit, noise_level: float = 0.01) -> float:
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit with depolarizing noise."""
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    return cirq.DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix[0, 0].real


circuit: cirq.Circuit = benchmarks.generate_rb_circuits(n_qubits=1, num_cliffords=50)[0]

true_value = execute(circuit, noise_level=0.0)       # Ideal quantum computer.
noisy_value = execute(circuit)                       # Noisy quantum computer.
pec_value = pec.execute_with_pec(circuit, execute)   # Noisy quantum computer + Mitiq.

print(f"Error (w/o  Mitiq): %0.4f" %abs((true_value - noisy_value) / true_value))
print(f"Error (with Mitiq): %0.4f" %abs((true_value - pec_value) / true_value))

TypeError: execute_with_pec() missing 1 required keyword-only argument: 'representations'

In [19]:
from mitiq.pec import OperationRepresentation

In [30]:
from mitiq.pec.representations.depolarizing import represent_operation_with_local_depolarizing_noise

In [21]:
reps = represent_operations_in_circuit_with_local_depolarizing_noise(circuit, noise_level=0.01)

In [22]:
print(reps[0])

0: ───Y^0.5─── = 1.010*(0: ───Y^0.5───)-0.003*(0: ───Y^0.5───X───)-0.003*(0: ───Y^0.5───Y───)-0.003*(0: ───Y^0.5───Z───)


In [23]:
reps[0].basis_expansion

1.010*0: ───Y^0.5───-0.003*0: ───Y^0.5───X───-0.003*0: ───Y^0.5───Y───-0.003*0: ───Y^0.5───Z───

In [36]:
reps = [represent_operation_with_local_depolarizing_noise(cirq.Circuit(op), noise_level=0.001) for op in set(circuit.all_operations())]

In [39]:
type(reps[0])

mitiq.pec.types.types.OperationRepresentation

In [ ]:
pec_